### This notebook is meant to create appropriate data for the subject wise engagement analysis. It creates a seperate dataframe for raw metrics, unnormalized metrics and log-normalized metrics. This data will be used to create vizualizations in the web-app

### **NOTE: At the end of notebook, we also have done similar analysis for overall discourse engagement.**

In [ ]:
import pandas as pd
import numpy as np
import os
from global_functions_1 import sanitize_filepath # function to sanitize file paths

# Creating metrics for each subject

In [3]:
action_to_description = {
"1": "likes_given",
"2": "likes_received",
"3": "bookmarked_post",
"4": "created_new_topic",
"5": "replied",
"6": "received_response",
"7": "user_was_mentioned",
"9": "user's_post_quoted",
"11": "user_edited_post",
"12": "user_sent_private_message",
"13": "recieved_a_private_message",
"15": "solved_a_topic",
"16": "user_was_assigned",
"17": "linked"
}
def create_raw_metrics_dataframe(df):
    # Change the values in action_name column based on values of action_type and map it via the action_to_description dictionary. This is done to make the column_names more intuitive to understand.
    subject_dataframe = df.copy()
    subject_dataframe['action_type'] = subject_dataframe['action_type'].astype(str)
    subject_dataframe['action_name'] = subject_dataframe['action_type'].map(action_to_description)
    subject_dataframe = pd.crosstab(df["acting_username"], subject_dataframe["action_name"]) # Creating PIVOT table

    columns_to_be_dropped = ['linked','received_response', "user's_post_quoted",
        'user_edited_post', 'user_was_mentioned'] # dropping columns which are not required for analysis

    subject_dataframe.drop(columns_to_be_dropped, axis=1, inplace=True, errors='ignore')
    
    subject_dataframe['acting_username'] = subject_dataframe.index # Changing the index to a column
    subject_dataframe = subject_dataframe[["acting_username"]+[col for col in subject_dataframe.columns if col != 'acting_username']]  # Reordering the columns
    subject_dataframe.index = range(0, len(subject_dataframe))
    subject_dataframe.columns.name = None
    return subject_dataframe # Returns raw metrics dataframe

In [29]:
# Assign the weights to the relevant columns. This can be changed as per the requirement.
weights_dict = { 'likes_given': 0.3, # 0.3
                "likes_received": 0.8, # changed from 0.7
                "created_new_topic": 0.5, # changed from 1.0
                "replied": 0.7,
                'solved_a_topic': 10 # Highest weight
}

def create_raw_scores_dataframe(df): # unnormalised scores
    df2 = pd.DataFrame(df.copy())
    columns_to_be_ignored = ["initial_score",'username','overall_topics_count_of_this_subject', 'normalised_score', 'z_score', "acting_username"] # If some column names seem irrelevant, please ignore them.

    df2["initial_score"] = sum(df2[column]*weights_dict[column] for column in df2.columns if column not in columns_to_be_ignored) # Initial score = sum(column_value*weight)

    df2["z_score"] = round((df2["initial_score"] - df2["initial_score"].mean()) / df2["initial_score"].std(),2) # z_score rounded to 2 decimal places
    return df2.sort_values(by="z_score",ascending=False)

def create_log_normalized_scores_dataframe(df):
    # Apply log normalization to the numerical features
    numerical_features = list(weights_dict.keys())
    # print(f"numerical_features on which log_normalization is applied: {numerical_features}")
    log_normalized_dataframe = df.copy()
    for feature in numerical_features:
        try:
            log_normalized_dataframe[feature] = round(np.log1p(log_normalized_dataframe[feature]),3)
        except Exception as e:
            print(f"\n********Error in log normalization for feature {feature}: {e}\n********\n")
            numerical_features.remove(feature) # Remove the feature from the list if log normalization fails
            continue
    
    log_normalized_dataframe["initial_score"] = log_normalized_dataframe[numerical_features].sum(axis=1)
    log_normalized_dataframe["z_score"] = round(
        (log_normalized_dataframe["initial_score"] - log_normalized_dataframe["initial_score"].mean()) / log_normalized_dataframe["initial_score"].std(),2)
    return log_normalized_dataframe.sort_values(by="z_score",ascending=False)


In [30]:
# Get all the category_IDs
all_category_ids = pd.read_csv("../data/all_category_ids.csv")
all_category_ids.head()

,category_id,name
0,18,Mathematics for Data Science I
1,19,Statistics for Data Science I
2,20,Computational Thinking
3,21,English I
4,22,English II


In [32]:
terms_folder = "../data/course_specific_data"
for term in list(os.listdir(terms_folder))[1:]:
    for item in all_category_ids.itertuples(name=None):
        # try:
            # Now process the data for each course in the term
            category_id = item[1]
            category_name = sanitize_filepath(item[2])
            print(f"Processing {category_name} for term {term}")
            
            # Read the user_actions_data for the course
            user_actions_dataframe = pd.read_excel(f"../data/course_specific_data/{term}/{category_name}.xlsx", sheet_name="user_actions_data")

            # If some course is not offered in a term, the user_action_dataframe can be empty or it can have very less rows (because of some overlapping days in the term). We will process the data iff it has more than 75 rows.
            if (not user_actions_dataframe.empty) and (len(user_actions_dataframe) > 75):          
                raw_metrics_dataframe = create_raw_metrics_dataframe(user_actions_dataframe)
                raw_scores_dataframe = create_raw_scores_dataframe(raw_metrics_dataframe) # unnormalised scores
                log_normalized_scores_dataframe = create_log_normalized_scores_dataframe(raw_scores_dataframe) # log normalized scores
            else:
                print(f"Making empty dataframes for {category_name} as it has <= {len(user_actions_dataframe)} rows")
                raw_metrics_dataframe, raw_scores_dataframe, log_normalized_scores_dataframe = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
            

            scores_data_path = f"../data/scores/{term}" # The scores for each course will be stored for each term in a separate folder.
            os.makedirs(scores_data_path, exist_ok=True)
            file_path = f"{scores_data_path}/{category_name}.xlsx"

            # Use ExcelWriter in 'openpyxl' mode to append multiple sheets
            with pd.ExcelWriter(file_path, engine="openpyxl", mode="w") as writer:
                raw_metrics_dataframe.to_excel(writer, sheet_name="raw_metrics", index=False)
                raw_scores_dataframe.to_excel(writer, sheet_name="unnormalized_scores", index=False)
                log_normalized_scores_dataframe.to_excel(writer, sheet_name="log_normalized_scores", index=False)
        # except Exception as e:
        #     print(f"Error: {e} for category_name: {category_name}")
        #     continue


Processing Mathematics_for_Data_Science_I for term t1_2025
Processing Statistics_for_Data_Science_I for term t1_2025
Processing Computational_Thinking for term t1_2025
Processing English_I for term t1_2025
Processing English_II for term t1_2025
Processing Mathematics_for_Data_Science_II for term t1_2025
Processing Statistics_for_Data_Science_II for term t1_2025
Processing Programming_in_Python for term t1_2025
Processing Programming,_Data_Structures_and_Algorithms for term t1_2025
Processing Database_Management_Systems for term t1_2025
Processing Machine_Learning_Foundations for term t1_2025
Processing Modern_Application_Development_I for term t1_2025
Processing Business_Data_Management for term t1_2025
Processing Machine_Learning_Techniques for term t1_2025
Processing Machine_Learning_Practice for term t1_2025
Processing Tools_in_Data_Science for term t1_2025
Processing Business_Analytics for term t1_2025
Processing System_Commands for term t1_2025
Processing Modern_Application_Develo

# Analysis of "all users engagement"

In [33]:
# Assign the weights to the relevant columns. This can be changed as per the requirement.
weights_dict = { 'likes_given': 0.4, # likes_given is also important
                "likes_received": 0.8,
                "topics_created": 0.4,
                "posts_created": 0.7,
                "days_visited": 0.3, # decreased weightage because it is a very common action
                'solutions': 10,
                "cheers": 0, # discard cheers as it's meaning is not clear.
}

def create_raw_scores_dataframe_for_all_users(df_original):
    df = df_original.copy()
    df["initial_score"] = sum(df[column]*weights_dict[column] for column in df.columns if column not in ["user_id"]) # Initial score = sum(column_value*weight)

    df["z_score"] = round((df["initial_score"] - df["initial_score"].mean()) / df["initial_score"].std(),2) # z_score rounded to 2 decimal places
    return df.sort_values(by="z_score",ascending=False)

def create_log_normalized_scores_dataframe(df):
    # Apply log normalization to the numerical features
    numerical_features = list(weights_dict.keys())
    # print(f"numerical_features on which log_normalization is applied: {numerical_features}")
    log_normalized_dataframe = df.copy()
    for feature in numerical_features:
        log_normalized_dataframe[feature] = round(np.log1p(log_normalized_dataframe[feature]),3)
    
    log_normalized_dataframe["initial_score"] = log_normalized_dataframe[numerical_features].sum(axis=1)
    log_normalized_dataframe["z_score"] = round((log_normalized_dataframe["initial_score"] - log_normalized_dataframe["initial_score"].mean()) / log_normalized_dataframe["initial_score"].std(),2)
    return log_normalized_dataframe.sort_values(by="z_score",ascending=False)

In [37]:
overall_scores_folder = "../data/scores/overall_scores" # This will contain the scores of users which will reflect their overall discourse engagement in that particular term
os.makedirs(overall_scores_folder, exist_ok=True)

raw_data_for_all_users = "../data/overall_data_all_users"

for file in os.listdir(raw_data_for_all_users):
    term = file.split("data_all_users_")[1].removesuffix(".xlsx")
    file_path = os.path.join(raw_data_for_all_users,file)
    user_actions_dataframe = pd.read_excel(file_path, sheet_name="user_actions_data").drop(columns=["posts_read","topics_viewed","cheers"])

    raw_scores_dataframe = create_raw_scores_dataframe_for_all_users(user_actions_dataframe)
    # log_normalized_scores_dataframe = create_log_normalized_scores_dataframe(raw_metrics_dataframe)
    final_metrics_path = os.path.join(overall_scores_folder,term)
    print(final_metrics_path)
    with pd.ExcelWriter(f"{final_metrics_path}.xlsx", engine="openpyxl", mode="w") as writer:
        # raw_metrics_dataframe.to_excel(writer, sheet_name="raw_metrics", index=False)
        raw_scores_dataframe.to_excel(writer, sheet_name="unnormalized_scores", index=False)
        # log_normalized_scores_dataframe.to_excel(writer, sheet_name="log_normalized_scores", index=False)
        # We have not taken log normalized scores for all users as it is not required for now.

../data/scores/overall_scores\t1_2024
../data/scores/overall_scores\t1_2025
../data/scores/overall_scores\t2_2024
../data/scores/overall_scores\t3_2024


# TESTING/EXPLORING

In [ ]:
# df = pd.read_excel("course_excel_data_t1_2024/English II.xlsx", sheet_name="unnormalized_scores").head()
# df

,acting_username,created_new_topic,likes_given,likes_received,replied,solved_a_topic,initial_score,z_score
0,teza,0,0,10,21,7,29.7,6.64
1,23f2002361,7,1,3,9,0,12.5,2.52
2,23f1000917,2,2,6,8,0,12.0,2.40
3,23f3001021,2,2,3,5,1,8.5,1.57
4,23f3003974,1,3,3,2,0,5.2,0.78


In [ ]:
# create_log_normalized_scores_dataframe(df)

,acting_username,created_new_topic,likes_given,likes_received,replied,solved_a_topic,initial_score,z_score
0,teza,0.000,0.000,2.398,3.091,2.079,7.568,1.27
1,23f2002361,2.079,0.693,1.386,2.303,0.000,6.461,0.24
2,23f1000917,1.099,1.099,1.946,2.197,0.000,6.341,0.13
3,23f3001021,1.099,1.099,1.386,1.792,0.693,6.069,-0.12
4,23f3003974,0.693,1.386,1.386,1.099,0.000,4.564,-1.52


In [6]:
path = "../data/course_specific_data/t1_2024/Advanced_Algorithms.xlsx"
df = pd.read_excel(path, sheet_name="user_actions_data")

In [8]:
df.empty

True